# Overview

This notebook outlines a workflow and executes key code for batch processing of autotracking.



# Paths, packages, and parameters
You'll need to execute the next cell for all of the code that follows. 

Make sure that the root_path and local_path are correct for your system.

The root_path is the path to the folder containing the data, and video folders.

local_path needs to be a directory on a local drive for writing binary video files for TGrabs/TRex.

In [1]:
# The project name need to match a directory name within the root path
proj_name = 'RN_Prop'

# Other details about the project
species    = 'rummy_nose'
exp_type   = 'prop_neo'

# font size for GUIs
font_size = 30

# Repeated measures for each calibration video
num_reps = 3

# Max number of frames for the mean image
max_num_frame_meanimage = 300

# Raw and processed video extensions
vid_ext_raw = 'MOV'
vid_ext_proc = 'mp4'

# Installed packages
import os
import platform
import numpy as np
import pandas as pd
import cv2

# Our own modules
import def_acquisition as da
import def_paths as dp
import video_preprocess as vp
import acqfunctions as af
import gui_functions as gf

# DEFINE ROOT PATH ============================================================

# Matt's laptop
if (platform.system() == 'Darwin') and (os.path.expanduser('~')=='/Users/mmchenry'):
    
    root_path = '/Users/mmchenry/Documents/Projects/waketracking'

# Matt on PopOS! machine
elif (platform.system() == 'Linux') and (os.path.expanduser('~')=='/home/mmchenry'):

    # root_path = '/home/mmchenry/Documents/wake_tracking'
    root_path = '/mnt/schooling/TRex'
    local_path = '/home/mmchenry/Documents/wake_tracking/video/binary'

# Ashley on Linux
elif (platform.system() == 'Linux') and (os.path.expanduser('~')=='/home/anpetey'):

    root_path = '/vortex/schooling/TRex'
    local_path = '/home/anpetey/Documents/wake_tracking/video/binary'

# Catch alternatives
else:
    raise ValueError('Do not recognize this account -- add lines of code to define paths here')

# =============================================================================

# Check for local path definition
if not 'local_path' in locals():
    raise ValueError('Local path not defined')

# Check paths
if not os.path.exists(root_path):
    raise ValueError('Root path does not exist: ' + root_path)
elif not os.path.exists(local_path):
    raise ValueError('Local path does not exist: ' + local_path)

# Get paths 
path = dp.give_paths(root_path, proj_name)

# Function that generates a filename
def generate_filename(date, sch_num, trial_num=None):
    if trial_num is None:
        return date + '_sch' + str(int(sch_num)).zfill(3)
    else:
        return date + '_sch' + str(int(sch_num)).zfill(3) + '_tr' + str(int(trial_num)).zfill(3)

# If using Jupyter notebook outside of VS Code, enable auto-reload
# %load_ext autoreload
# %autoreload 2

# Interactive video measurements
This section interactively prompts a user for what's needed to preprocess videos from a particular schedule. It prompts the user for information that it needs to create a mask, perform a spatial calibration, and select threshold and blob area values for the image processing by TGrabs and TRex.

## Select schedule, check for problems in recordings
Note: need to run this for cells below.

Here we prompt the user to select which schedule to choose for preprocessing. Along the way, it checks for the following:
- That the experiment_log.csv and recording_log.csv lists include all trials in the schedule.
- Compares the schedules in the project against video recordings 
- It compares the duration of recorded videos to what was expected in the schedule and alerts user of large differences.

In [2]:
# Find matching directories between the schedule and video directories
matching_sch, nonmatching_sch = vp.find_schedule_matches(path['sch'], path['vidin'])

# if matching_directories is empty, then say so and exit
if len(matching_sch) == 0:
    print(' ')
    print("No matching directories found between the dates in the schedule list and the dates in the video directory.")
    print('Schedule directory:',  path['sch'])
    print('Video directory:',     path['vidin'])
    print(' ')
    sys.exit()

# If there are matches . . .
else:
    # if nonmatching_directories is not empty, then print the list of nonmatching directories
    if len(nonmatching_sch) > 0:
        print("Note that the following schedules do not have matching dates in the video directory:")
        for directory in nonmatching_sch:
            print('   ' + directory)

    # Use the list of matching directories for user selection
    analysis_schedule = gf.select_item(matching_sch, 'Select which schedule to work on', font_size=font_size)

    # define sch_num as the number given from the last two characters of analysis_schedule
    sch_num = int(analysis_schedule[-2:])

    # define sch_date as the date given from the first 10 characters of analysis_schedule
    sch_date = analysis_schedule[:10]

# Get schedule data
sch = pd.read_csv(path['sch'] + os.sep + analysis_schedule + '.csv')

# Extract experiment catalog info
cat = af.get_cat_info(path['cat'], include_mode='both', exclude_mode='calibration')
if len(cat) == 0:
    raise ValueError('No videos requested to work on from experiment_log.' + \
                     ' Both the columns \'analyze\' and \'make_video\' must be' + \
                     ' set to 1 for pre-processing.')

# Extract experiment log info
log = pd.read_csv(path['data'] + os.sep + 'recording_log.csv')

# Return a version of cat for all matches of the sch_num column that matches sch_num
cat_curr = cat[cat['sch_num'] == sch_num]

# Return a version of log for all matches of the sch_num column that matches sch_num and all matches of the sch_date column that matches sch_date
log_curr = log[(log['sch_num'] == sch_num) & (log['date'] == sch_date)]

# Check if the number of rows of the schedule matches the number of rows of the current catalog
if cat_curr.shape[0]!=sch.shape[0]:
    print(' ')
    print('WARNING: The number of rows of the schedule do not match the current catalog from experiment_log.csv.')

# Make list of any trial numbers in sch that do not match any trial num in cat_curr
missing_trials_cat = [trial for trial in sch['trial_num'] if trial not in cat_curr['trial_num'].values]
missing_trials_log = [trial for trial in sch['trial_num'] if trial not in log_curr['trial_num'].values]

# If there are missing trials, then print the list of missing trials
print(' ')
if len(missing_trials_cat) > 0:
    print("Note that the following trials in the schedule do not have matching videos in experiment_log.csv:")
    for trial in missing_trials_cat:
        print('   ' + str(trial))
else:
    print('All trials in the schedule have matching videos in experiment_log.csv')

# If there are missing trials, then print the list of missing trials
print(' ')
if len(missing_trials_log) > 0:
    print("Note that the following trials in the schedule do not have matching videos in recording_log.csv:")
    for trial in missing_trials_log:
        print('   ' + str(trial))
else:
    print('All trials in the schedule have matching videos in recording_log.csv')

# Path to all videos for the current date
vid_path = path['vidin'] + os.sep +  sch_date

# Flag any large differences in video duration from experiment log, return list of videos to be processed
vid_files = vp.check_video_duration(vid_path, sch, cat, vid_ext=vid_ext_raw, thresh_time=3.0)

# Read the full cat file
cat_raw = pd.read_csv(path['cat'])

# Check if any timecode values have not been specified
all_timecodes_specified = False
if 'timecode_start' in cat_raw.columns:
    # Filter the DataFrame based on the condition
    filtered_data = cat_raw[(cat_raw['date'] == sch_date) & (cat_raw['sch_num'] == sch_num)]

    # Check if all timecode values are specified
    all_timecodes_specified = filtered_data['timecode_start'].notnull().all()
    
# If there's any missing calibration values . . .
if True: #not all_timecodes_specified:
    # Add timecode data to cat_raw
    cat_raw = vp.add_start_timecodes(vid_files, vid_path, cat_raw)

    # Write cat_raw, if it has the same dimensions, or one new column
    cat_raw.to_csv(path['cat'], index=False)
    print(' ')
    print('Added time code data to experiment_log.csv')
else:
    print(' ')
    print('Time code data already exists in experiment_log.csv')


 
All trials in the schedule have matching videos in experiment_log.csv
 
All trials in the schedule have matching videos in recording_log.csv
 
Added time code data to experiment_log.csv


## Create a mask image
You will want to choose a region of interest that is just outside of the water line within the arena.

In [ ]:
# Define the mask filename
mask_filename = generate_filename(sch_date, sch_num, trial_num=None)
mask_path = path['mask'] + os.sep + mask_filename + '_mask.jpg'

# If the mask file does not exist, then create it
if True: #not os.path.exists(mask_path):
   centroid = gf.create_mask_for_batch(vid_path+os.sep+vid_files[0], mask_path)

   # Read the full cat file
   cat_raw = pd.read_csv(path['cat'])

   # Get the size of the cat_raw
   cat_raw_size = cat_raw.shape

   # Determine if roi centroid values already exsist in cat_raw where date==sch_date and sch_num==sch_num
   roi_x_exists = cat_raw.loc[(cat_raw['date'] == sch_date) & (cat_raw['sch_num'] == sch_num), 'roi_x'].values

   roi_y_exists = cat_raw.loc[(cat_raw['date'] == sch_date) & (cat_raw['sch_num'] == sch_num), 'roi_y'].values

   # If there's any missing roi values . . .
   if max(np.isnan(roi_x_exists)):
        # Find video_filename for calibration from cat_raw: where the date matches sch_date and the sch_num is 999
        cal_video_filename = cat_raw[(cat_raw['date'] == sch_date) & (cat_raw['sch_num'] == 999)]['video_filename'].values

        # Add roi centroid value to cat_raw.roi_x and cat_raw.roi_y where sch_num=sch_num
        cat_raw.loc[(cat_raw['date'] == sch_date) & (cat_raw['sch_num'] == sch_num), 'roi_x'] = centroid[0]
        cat_raw.loc[(cat_raw['date'] == sch_date) & (cat_raw['sch_num'] == sch_num), 'roi_y'] = centroid[1]

        # Write cat_raw, if it has the same dimensions, or one new column
        if (cat_raw.shape[0] == cat_raw_size[0]) and \
            (cat_raw.shape[1] == cat_raw_size[1]):
            cat_raw.to_csv(path['cat'], index=False)
            print(' ')
            print('Added roi centroid values to experiment_log.csv')
        else:
            # raise exception
            raise ValueError('cat_raw has the wrong dimensions-- cannot write the roi centroid data to experiment_log')


else:
    print(' ')
    print('Mask file and roi already exists. Using existing mask file: ' + mask_path)   

## Run spatial calibration
Prompts user to conduct repeated measures for the calibration. Note that you need to know the actual length in centimeters.

In [ ]:
# Read the full cat file
cat_raw = pd.read_csv(path['cat'])

# Get the size of the cat_raw
cat_raw_size = cat_raw.shape

# Determine if cm_per_pix values already exsist in cat_raw where date==sch_date and sch_num==sch_num
cm_per_pix_exists = cat_raw.loc[(cat_raw['date'] == sch_date) & (cat_raw['sch_num'] == sch_num), 'cm_per_pix'].values

# If there's any missing calibration values . . .
if max(np.isnan(cm_per_pix_exists)):

    # Find video_filename for calibration from cat_raw: where the date matches sch_date and the sch_num is 999
    cal_video_filename = cat_raw[(cat_raw['date'] == sch_date) & (cat_raw['sch_num'] == 999)]['video_filename'].values

    # Define the full path to the calibration video
    full_vid_path = vid_path + os.sep + cal_video_filename[0] + '.' + vid_ext_raw

    # Raise exception if cal_video_filename has a length of zero
    if len(cal_video_filename) == 0:
        raise ValueError('The calibration video does not exist in the catalog file')
    # Or, more than one
    elif len(cal_video_filename) > 1:
        raise ValueError('More than one calibration video exists in the catalog file')

    # Raise exception if cal_video_filename is not in vid_path
    if not os.path.exists(full_vid_path):
        raise ValueError('The calibration video does not exist in the video directory: ' + full_vid_path)

    # Run the spatial calibration
    cm_per_pix = gf.run_spatial_calibration(full_vid_path, reps=3, font_size=font_size)

    # Add cm_per_pix value to cat_raw.cm_per_pix where sch_num=sch_num
    cat_raw.loc[(cat_raw['date'] == sch_date) & (cat_raw['sch_num'] == sch_num), 'cm_per_pix'] = cm_per_pix

    # Write cat_raw, if it has the same dimensions, or one new column
    if (cat_raw.shape[0] == cat_raw_size[0]) and \
        (cat_raw.shape[1] == cat_raw_size[1]):
        cat_raw.to_csv(path['cat'], index=False)
        print(' ')
        print('Added cm_per_pix values to experiment_log.csv')
    else:
        # raise exception
        raise ValueError('cat_raw has the wrong dimensions-- cannot write the time code data to experiment_log')
    
else:
    print(' ')
    print('cm_per_pix values already exist in experiment_log.csv for the current date and schedule number.')

## Create mean image
A mean image is created from multiple videos in the batch.

In [ ]:
# Define the mask filename
mask_filename = generate_filename(sch_date, sch_num, trial_num=None)
mask_path = path['mask'] + os.sep + mask_filename + '_mask.jpg'

# Mean image path
mean_image_path = path['mean'] + os.sep + mask_filename + '_mean.jpg'

# If the mean image does not exist, then create it
if not os.path.exists(mask_path):

    # Find the mask
    im_mask, mask_perim = vp.get_mask(mask_path)

    # Make mean image
    mean_image = vp.make_max_mean_image(cat_curr, sch, vid_path, max_num_frame_meanimage, im_mask=im_mask, mask_perim=mask_perim, im_crop=True)

    # Save the mean image
    mean_image_path = path['mean'] + os.sep + mask_filename + '_mean.jpg'
    cv2.imwrite(mean_image_path, mean_image)

# If the mean image does exist, then read it
else:
    # Read the mean image
    mean_image = cv2.imread(mean_image_path, cv2.IMREAD_UNCHANGED)

# Display the binary image
gf.create_cv_window('Mean image')
cv2.imshow('Mean image', mean_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

## Select threshold and blob area

- Select the lowest threshold possible, without the margins of each fish looking fuzzy

- Select the range of areas that just barely include individual fish. Exclude fish that are touching area other.

In [ ]:
# Read the full cat file
cat_raw = pd.read_csv(path['cat'])

# Get the size of the cat_raw
cat_raw_size = cat_raw.shape

# Check if 'min_area' column exists in cat_raw and does not have nans
if 'min_area' not in cat_raw.columns or cat_raw['min_area'].isnull().any():

    # Get the mask
    mask_filename = generate_filename(sch_date, sch_num, trial_num=None)
    mask_path = path['mask'] + os.sep + mask_filename + '_mask.jpg'
    im_mask, mask_perim = vp.get_mask(mask_path)

    # Get the mean image
    mean_image_path = path['mean'] + os.sep + mask_filename + '_mean.jpg'
    mean_image = cv2.imread(mean_image_path, cv2.IMREAD_UNCHANGED)

    # read first frame of first video
    vid_path_curr = vid_path + os.sep + vid_files[0]
    vid = cv2.VideoCapture(vid_path_curr)
    im_start = vp.read_frame(vid, 0, im_mask=im_mask, mask_perim=mask_perim, im_crop=True)
    vid.release()

    # Select the threshold
    threshold, im_thresholded = gf.interactive_threshold(im_start, mean_image)
    print('Selected threshold = ' + str(threshold))

    # Select the bounds of blob area
    print(' ')
    print('Select the bounds of blob area that include just a single fish')
    min_area, max_area = gf.interactive_blob_filter(im_start, mean_image, threshold)
    print('Selected min_area = ' + str(min_area))
    print('Selected max_area = ' + str(max_area))

    # Save results to experiment_log.csv
    cat_raw.loc[(cat_raw['date'] == sch_date) & (cat_raw['sch_num'] == sch_num), 'threshold'] = threshold
    cat_raw.loc[(cat_raw['date'] == sch_date) & (cat_raw['sch_num'] == sch_num), 'min_area'] = min_area
    cat_raw.loc[(cat_raw['date'] == sch_date) & (cat_raw['sch_num'] == sch_num), 'max_area'] = max_area

    # Write cat_raw, if it has the same dimensions, or one new column
    cat_raw.to_csv(path['cat'], index=False)
    print(' ')
    print('Added threshold and area values to experiment_log.csv')

else:
    print(' ')
    print('Threshold and area values already exist in experiment_log.csv for the current date and schedule number.')

## Generate binary videos

Here we use the threshold and area values to generate black-and-white images of the school.

This can be performed on a single video, all videos in a schedule in succession, or using parallel processing (the fastest option).

### Single video

In [3]:
# Get the mask
mask_filename = generate_filename(sch_date, sch_num, trial_num=None)
mask_path = path['mask'] + os.sep + mask_filename + '_mask.jpg'
im_mask, mask_perim = vp.get_mask(mask_path)

# Get the mean image
mean_image_path = path['mean'] + os.sep + mask_filename + '_mean.jpg'
mean_image = cv2.imread(mean_image_path, cv2.IMREAD_UNCHANGED)

# Single video address in cat_curr
index = 0
row = cat_curr.iloc[0]

# Paths for input and output videos
vid_path_in = vid_path + os.sep + row['video_filename'] + '.' + vid_ext_raw
vid_file_out = generate_filename(row['date'], row['sch_num'], trial_num=row['trial_num'])
vid_path_out = local_path + os.sep + vid_file_out + '.' + vid_ext_proc

# Set bounds of the area for blobs
min_area = int(row['min_area']/4)
max_area = int(10*row['max_area'])

print('Video in: '  + vid_path_in)
print('Video out: ' + vid_path_out)

status_txt = 'Trial ' + str(row['trial_num'])

# Generate and save binary movie
vp.make_binary_movie(vid_path_in, vid_path_out, mean_image, row['threshold'], min_area, max_area, \
                        im_mask=im_mask, mask_perim=mask_perim, im_crop=True, status_txt=status_txt, echo=True, blob_color='grayscale')

Video in: /vortex/schooling/TRex/video/RN_Prop/raw/2023-06-28/RNPROP73_S001_S001_T037.MOV
Video out: /home/anpetey/Documents/wake_tracking/video/binary/2023-06-28_sch001_tr001.mp4
   Trial 1.0: Finished frame 1 of 5372. Estimated time remaining: 69.0 min
   Trial 1.0 Frame 30 : Increasing threshold to 6
   Trial 1.0 Frame 30 : Decreasing threshold to 5
   Trial 1.0 Frame 65 : Increasing threshold to 6
   Trial 1.0 Frame 65 : Decreasing threshold to 5
   Trial 1.0 Frame 71 : Increasing threshold to 6
   Trial 1.0 Frame 71 : Decreasing threshold to 5
   Trial 1.0: Finished frame 101 of 5372. Estimated time remaining: 84.0 min
   Trial 1.0 Frame 111 : Increasing threshold to 6
   Trial 1.0 Frame 111 : Decreasing threshold to 5
   Trial 1.0 Frame 113 : Increasing threshold to 6
   Trial 1.0 Frame 114 : Decreasing threshold to 5
   Trial 1.0 Frame 163 : Increasing threshold to 6
   Trial 1.0 Frame 164 : Decreasing threshold to 5
   Trial 1.0: Finished frame 201 of 5372. Estimated time remai

KeyboardInterrupt: 

### One video at a time

In [4]:
# Get the mask
mask_filename = generate_filename(sch_date, sch_num, trial_num=None)
mask_path = path['mask'] + os.sep + mask_filename + '_mask.jpg'
im_mask, mask_perim = vp.get_mask(mask_path)

# Get the mean image
mean_image_path = path['mean'] + os.sep + mask_filename + '_mean.jpg'
mean_image = cv2.imread(mean_image_path, cv2.IMREAD_UNCHANGED)

# Loop thru each row of cat_curr
for index, row in cat_curr.iterrows():

    # Paths for input and output videos
    vid_path_in = vid_path + os.sep + row['video_filename'] + '.' + vid_ext_raw
    vid_file_out = generate_filename(row['date'], row['sch_num'], trial_num=row['trial_num'])
    vid_path_out = local_path + os.sep + vid_file_out + '.' + vid_ext_proc

    # Set bounds of the area for blobs
    min_area = int(row['min_area']/4)
    max_area = int(10*row['max_area'])

    print('Video in: '  + vid_path_in)
    print('Video out: ' + vid_path_out)

    status_txt = 'Trial ' + str(row['trial_num'])

    # Generate and save binary movie
    vp.make_binary_movie(vid_path_in, vid_path_out, mean_image, row['threshold'], min_area, max_area,
                         im_mask=im_mask, mask_perim=mask_perim, im_crop=True, status_txt=status_txt, echo=True, blob_color='white')

Video in: /vortex/schooling/TRex/video/RN_Prop/raw/2023-06-28/RNPROP73_S001_S001_T037.MOV
Video out: /home/anpetey/Documents/wake_tracking/video/binary/2023-06-28_sch001_tr001.mp4
   Trial 1.0: Finished frame 1 of 5372. Estimated time remaining: 68.5 min
   Trial 1.0 Frame 30 : Increasing threshold to 6
   Trial 1.0 Frame 30 : Decreasing threshold to 5
   Trial 1.0 Frame 65 : Increasing threshold to 6
   Trial 1.0 Frame 65 : Decreasing threshold to 5
   Trial 1.0 Frame 71 : Increasing threshold to 6
   Trial 1.0 Frame 71 : Decreasing threshold to 5
   Trial 1.0: Finished frame 101 of 5372. Estimated time remaining: 84.0 min
   Trial 1.0 Frame 111 : Increasing threshold to 6
   Trial 1.0 Frame 111 : Decreasing threshold to 5
   Trial 1.0 Frame 113 : Increasing threshold to 6
   Trial 1.0 Frame 114 : Decreasing threshold to 5
   Trial 1.0 Frame 163 : Increasing threshold to 6
   Trial 1.0 Frame 164 : Decreasing threshold to 5
   Trial 1.0: Finished frame 201 of 5372. Estimated time remai

### Parallel processing

In [ ]:
import concurrent.futures
import time

# Record the start time
start_time = time.time()

# Get the mask
mask_filename = generate_filename(sch_date, sch_num, trial_num=None)
mask_path = path['mask'] + os.sep + mask_filename + '_mask.jpg'
im_mask, mask_perim = vp.get_mask(mask_path)

# Get the mean image
mean_image_path = path['mean'] + os.sep + mask_filename + '_mean.jpg'
mean_image = cv2.imread(mean_image_path, cv2.IMREAD_UNCHANGED)

# Define a function to process each row in parallel
def process_row(row):
    # Paths for input and output videos
    vid_path_in = vid_path + os.sep + row['video_filename'] + '.' + vid_ext_raw
    vid_file_out = generate_filename(row['date'], row['sch_num'], trial_num=row['trial_num'])
    vid_path_out = local_path + os.sep + vid_file_out + '.' + vid_ext_proc

    # Set bounds of the area for blobs
    min_area = int(row['min_area'] / 4)
    max_area = int(10*row['max_area'])

    status_txt = 'Trial ' + str(row['trial_num'])

    # Generate and save binary movie
    vp.make_binary_movie(vid_path_in, vid_path_out, mean_image, row['threshold'], min_area, max_area,
        im_mask=im_mask, mask_perim=mask_perim, im_crop=True, status_txt=status_txt, echo=True, blob_color='grayscale')

# Create a ThreadPoolExecutor to execute the iterations in parallel
with concurrent.futures.ThreadPoolExecutor() as executor:
    # Loop thru each row of cat_curr and submit each iteration as a separate task
    futures = [executor.submit(process_row, row) for _, row in cat_curr.iterrows()]

    # Wait for all tasks to complete
    concurrent.futures.wait(futures)

# Calculate the total execution time
execution_time = (time.time() - start_time)/60/60
print("Total execution time: {:.2f} hours".format(execution_time))

# tRex and tGrabs


## Parameters

Parameters are described in the documentation for [TGrabs](https://trex.run/docs/parameters_tgrabs.html) and [TRex](https://trex.run/docs/parameters_trex.html).

These lists of parameters will be passed to TGrabs and TRex. If there is not already a column for that parameter, then it will be added to cat (i.e. experiment_log.csv) with the default values specified below. Those defaults may be overridden by keying values into experiment_log.csv.

In [8]:
# Parameter list to use by TGrabs, along with default
param_list_tgrabs = [
    #['threshold','20'],
    ['averaging_method','mode'],
    ['average_samples','150'],
    ['blob_size_range','[0.0001,5000000]'],
    ['meta_conditions',exp_type],
    ['meta_species',species]
   # ['meta_misc','school_ABC']
    ]

# Specify list of parameter values for TRex, all listed as strings
param_list_trex = [
    ['track_threshold','1'],
    ['blob_size_ranges','[0.01,1.5]'],
    ['track_max_speed','70'],
    ['output_format','npz'],
    ['output_invalid_value','nan'],
    # ['gui_zoom_limit','[100,100]'],
    ['gui_show_posture','false'],
    ['gui_show_paths','false'],
    ['gui_show_outline', 'true'], 
    ['gui_show_midline', 'true'], 
    ['gui_show_blobs', 'true'],
    ['calculate_posture','true'],
    ['gui_show_number_individuals', 'true']
    ]

# Map 'cat' column names to TRex parameter names (no default values)
cat_to_trex = [
    ['fish_num','track_max_individuals'],
    ['cm_per_pix','cm_per_pixel'],
    ['frame_rate','frame_rate']
    ]

# Add default parameter values to all rows
af.add_param_vals(path['cat'], param_list_tgrabs, param_list_trex)

No new parameters added to cat file: /vortex/schooling/TRex/data/RN_Prop/experiment_log.csv


## Run TGrabs

TGrabs generates pv video files from raw videos for TRex tracking.

Cell below generates dv videos, which will be used by TRex, from compressed videos.
This will be completed for each row of cat.

In [11]:
# Run TGrabs, or formulate the command-line terminal commands
commands = af.run_tgrabs(path['cat'], local_path, path['vidpv'], param_list_tgrabs, vid_ext_proc=vid_ext_proc, use_settings_file=False, run_gui=True, echo=True, run_command=False)


 
TGrabs command (not run):
tgrabs -i /home/anpetey/Documents/wake_tracking/video/binary/2023-06-28_sch001_tr001.mp4 -o /vortex/schooling/TRex/video/RN_Prop/pv/2023-06-28_sch001_tr001.pv -nowindow false -track_max_individuals 30 -averaging_method mode -average_samples 150 -blob_size_range [0.0001,5000000] -meta_conditions prop_neo -meta_species rummy_nose 
 
TGrabs command (not run):
tgrabs -i /home/anpetey/Documents/wake_tracking/video/binary/2023-06-28_sch001_tr002.mp4 -o /vortex/schooling/TRex/video/RN_Prop/pv/2023-06-28_sch001_tr002.pv -nowindow false -track_max_individuals 30 -averaging_method mode -average_samples 150 -blob_size_range [0.0001,5000000] -meta_conditions prop_neo -meta_species rummy_nose 
 
TGrabs command (not run):
tgrabs -i /home/anpetey/Documents/wake_tracking/video/binary/2023-06-28_sch001_tr003.mp4 -o /vortex/schooling/TRex/video/RN_Prop/pv/2023-06-28_sch001_tr003.pv -nowindow false -track_max_individuals 30 -averaging_method mode -average_samples 150 -blob_siz

## Run TRex
Uses the parameter names given in param_list_trex and cat_to_trex to generate the command-line terminal commands to run TRex.


In [ ]:
    # Run TRex, or formulate the command-line terminal commands
commands = af.run_trex(path['cat'], path['vidpv'], path['data_raw'], param_list_trex, cat_to_trex, run_gui=True, output_posture=True, echo=True, run_command=False)

## Export TRex data in mat format

In [ ]:
# Extract experiment catalog info
cat = af.get_cat_info(path['cat'], include_mode='matlab', exclude_mode='calibration')

# Convert all npz files for an experiment to mat files.
da.raw_to_mat(cat, path)

# Housecleaning

In [ ]:
# Delete the local binary videos that match the pv videos
af.delete_matching_files(local_path, path['vidpv'])
git